<a href="https://colab.research.google.com/github/nupursjsu/Advanced-Deep-Learning/blob/master/Project/Breast_Cancer_Attention_Based(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scipy==1.2.0
from scipy.misc import imsave

     |████████████████████████████████| 26.6MB 68.7MB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [2]:
!gdown --id 1kLevdRmyykFfaiUt0A3uSFCRog-mOWSJ

Downloading...
From: https://drive.google.com/uc?id=1kLevdRmyykFfaiUt0A3uSFCRog-mOWSJ
To: /content/combined.zip
4.21GB [01:02, 67.4MB/s]


In [3]:
import zipfile
with zipfile.ZipFile('/content/combined.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/combined')

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Nov 23 01:42:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 13.3MB/s 


In [6]:
import os
import glob
from PIL import Image
import numpy as np
import torch
from torchvision import datasets, transforms
import random
from tqdm.autonotebook import tqdm
# dir structure would be: data/class_name(0 and 1)/dir_containing_img/img
class PatchMethod(torch.utils.data.Dataset):
    def __init__(self, root = 'Desktop/screenshots/', mode = 'train', transform = None):
        self.root = root
        self.mode = mode
        self.raw_samples = glob.glob(root + '/*/*')
        # print(self.raw_samples)
        self.samples = []
        for raw_sample in self.raw_samples:
            self.samples.append((raw_sample, int(raw_sample.split('/')[-2])))
            # print(raw_sample,int(raw_sample.split('/')[-2]))
        # print(self.samples)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        if self.mode == 'train':
            random.shuffle(self.samples)
            
        image_dir, label = self.samples[index]
        images = glob.glob(image_dir)
        # images = glob.glob(image_dir + '/*')
        
        t = transforms.Compose([transforms.CenterCrop((448,700))]) #centercropping to 1200 to generate 9 400x400 patches
        
        transformations = transforms.Compose([
            transforms.ToTensor()
        ])
        
        array = []
        
        for i, image_path in enumerate(images):
            # print(image_path)
            image = Image.open(image_path)
            image = np.array(t(image))
            # print(image.shape)
            # image = np.array(image)
            r, c, _ = image.shape
            # print("image.shape",image.shape)
            for i in range(0,28*16,28):
                for j in range(0,28*25,28):
                    array.append(transformations(image[i:i+28, j:j+28, :]))
#                     array.append(transformations(image[i:i+400, j:j+400, :]).float())
#                     array.append(image[i:i+400, j:j+400, :])
                    # if (i+400 < r) and (j+400 < c):
                        # array.append(transformations(image[i:i+400, j:j+400, :]).float())
                        # array.append(image[i:i+400, j:j+400, :])
                    
                    
        array = tuple(array)
        # print("################### array ###################")
        # print(array)
        array = torch.stack(array, 0)
        
        return (array, label)

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1
        self.feature_extractor_part1 = nn.Sequential(
            
            # torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            nn.Conv2d(3, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # print('in forward')
        # print('========================')
        # print(x.shape)
        x = x.squeeze(0)
        # print(x.shape)
        # print('========================')
        # print(x)
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 50 * 4 * 4)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        # error = 1. - Y_hat.eq(Y).cpu().float().mean().data[0]
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        # print(X,Y)
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

In [53]:
data_path_train='/content/combined/combined/train'
data_path_test='/content/combined/combined/test'
data = PatchMethod(root = data_path_train)
val_data = PatchMethod(root = data_path_test)

In [54]:

import numpy as np
# from scipy.misc import imsave
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable

# from dataloader import MnistBags
# from amil_model import Attention

# from __future__ import print_function, division
import os
import glob
from PIL import Image
import numpy as np
from torchvision import datasets, transforms
from torchvision import models
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter
import argparse

In [55]:
train_loader = torch.utils.data.DataLoader(data, shuffle = True, batch_size = 1
                                )
test_loader = torch.utils.data.DataLoader(val_data, shuffle = False, num_workers = 6, batch_size = 1)

val_loader = torch.utils.data.DataLoader(val_data, shuffle = False, num_workers = 6, batch_size = 1)

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Attention().to(device)

In [57]:
optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), weight_decay=10e-5)

In [58]:
def train(epoch):
    # progress = tqdm(enumerate(train_loader), desc="Loss: ", total=256)
    
    model.train()
    
    train_loss = 0.
    train_error = 0.
    correct_label_pred = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        
        
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        data = data.squeeze(0)

        optimizer.zero_grad()
        # calculate loss and metrics
        
        loss, _ = model.calculate_objective(data, bag_label)
        train_loss += loss.data[0]
        error, predicted_label_train = model.calculate_classification_error(data, bag_label)
        # print("bag_label,predicted_label_train",bag_label,predicted_label_train)
        # print(int(bag_label) == int(predicted_label_train))
        correct_label_pred += (int(bag_label) == int(predicted_label_train))
        # exit()
        train_error += error
        # backward pass
        loss.backward()
        # step
        optimizer.step()
        # print(correct_label_pred)
        # print(len(train_loader))
        # print(batch_idx)
      
    # calculate loss and error for epoch
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    train_acc = (1 - train_error)*100

    writer.add_scalar('data/train_acc', train_acc, epoch)
    writer.add_scalar('data/train_error', train_error, epoch)
    writer.add_scalar('data/train_loss', train_loss, epoch)

    result_train = 'Epoch: {}, Loss: {:.4f}, Train error: {:.4f}, Train accuracy: {:.2f}'.format(epoch, train_loss.cpu().numpy()[0], train_error, train_acc)

    print(result_train)
    return result_train

def test(epoch):
    model.eval()
    test_loss = 0.
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        # print(label)
        # print((data[0].shape))

        bag_label = label[0]
        instance_labels = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, attention_weights = model.calculate_objective(data, bag_label)
        test_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data, bag_label)
        test_error += error

        visualization_attention(data[0],attention_weights[0],batch_idx,epoch)
        if batch_idx < 1:  # plot bag labels and instance labels for first 5 bags
            bag_level = (bag_label.cpu().data.numpy(), int(predicted_label.cpu().data.numpy()))
            # print(bag_level)
            # print(instance_labels)
            # visualization_attention(data[0],attention_weights[0],batch_idx,epoch)
            # print("attention_weights.shape",attention_weights.shape)
            # instance_level = list(zip(instance_labels.numpy().tolist(),
            #                      np.round(attention_weights.cpu().data.numpy(), decimals=3).tolist()))

            # print('\nTrue Bag Label, Predicted Bag Label: {}\n'
            #       'True Instance Labels, Attention Weights: {}'.format(bag_level, instance_level))

    test_error /= len(test_loader)
    test_loss /= len(test_loader)
    test_acc = (1 - test_error)*100    

    writer.add_scalar('data/test_acc', test_acc, epoch)
    writer.add_scalar('data/test_error', test_error, epoch)
    writer.add_scalar('data/test_loss', test_loss, epoch)
    result_test = 'Epoch: {}, Loss: {:.4f}, test error: {:.4f}, test accuracy: {:.2f}'.format(epoch, test_loss.cpu().numpy()[0], test_error, test_acc)
    print(result_test)
    return result_test
    # print('Test Set, Loss: {:.4f}, Test error: {:.4f}'.format(test_loss.cpu().numpy()[0], test_error))

def visualization_attention(data,attention_weights,batch_idx,epoch):
    img_save_dir = './AMIL_visualization/epoch_{}'.format(epoch)
    img_save_name = img_save_dir + '/test_epoch_{}_no_{}.png'.format(epoch,batch_idx)
    if not os.path.exists(img_save_dir):
        os.makedirs(img_save_dir)

    data = data.cpu().data.numpy()
    attention_weights = attention_weights.cpu().data.numpy()
    # print("data.shape",data.shape)
    # print("attention_weights",attention_weights.shape)
    attention_weights = attention_weights/np.max(attention_weights)
    complete_image=np.zeros((3,480,700))
    for height_no in range(16):
        for width_no in range(25):
            complete_image[:,height_no*28:height_no*28+28, width_no*28:width_no*28+28] = data[height_no*25+width_no,:,:,:] * attention_weights[height_no*25+width_no]
    complete_image = complete_image.transpose((1,2,0))
    imsave(img_save_name,complete_image)
    # weighted_images_list = data * attention_weights


In [59]:
save_name_txt = "save_string.txt"
writer = SummaryWriter()
model_file = open(save_name_txt,"w") 
for epoch in range(1, 5 + 1):
        print('----------Start Training----------')
        train_result = train(epoch)
        print('----------Start Testing----------')
        test_result = test(epoch)
        model_file.write(test_result + '\n')
        model_file.write(train_result + '\n')
model_file.close()
torch.save(model.state_dict(),"AMIL_Breakthis_state_dict.pt")
torch.save(model ,"AMIL_Breakthis_model.pt")

----------Start Training----------
Epoch: 1, Loss: 4.3904, Train error: 0.3882, Train accuracy: 61.18
----------Start Testing----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch: 1, Loss: 7.8903, test error: 0.6854, test accuracy: 31.46
----------Start Training----------
Epoch: 2, Loss: 7.8744, Train error: 0.6840, Train accuracy: 31.60
----------Start Testing----------
Epoch: 2, Loss: 7.9198, test error: 0.6880, test accuracy: 31.20
----------Start Training----------
Epoch: 3, Loss: 5.4338, Train error: 0.4763, Train accuracy: 52.37
----------Start Testing----------
Epoch: 3, Loss: 3.5849, test error: 0.3114, test accuracy: 68.86
----------Start Training----------
Epoch: 4, Loss: 1.4049, Train error: 0.3139, Train accuracy: 68.61
----------Start Testing----------
Epoch: 4, Loss: 0.6363, test error: 0.3318, test accuracy: 66.82
----------Start Training----------
Epoch: 5, Loss: 3.6044, Train error: 0.3623, Train accuracy: 63.77
----------Start Testing----------
Epoch: 5, Loss: 7.9198, test error: 0.6880, test accuracy: 31.20
